In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'
from bkanalysis.tax import nutmeg

In [ ]:
# Asset Code is a SEDOL
IN_USD = True

In [ ]:
df_activity = pd.read_csv(r'', parse_dates=['Date'])

In [ ]:
df_piv = nutmeg.clean_nutmeg_activity_report(df_activity)

if IN_USD:
    df_gbpusd = pd.read_csv(r'', parse_dates=['Date'])
    gbpusd_close = df_gbpusd.set_index('Date')['Close']

    df_piv['fx'] = [gbpusd_close[d] for d in df_piv.index]
    df_piv['Unitary Value'] = df_piv['Unitary Value'] * df_piv['fx']

In [ ]:
tax_tables = []
for asset_code in df_piv['Asset Code'].unique():
    if asset_code == 'CASH':
        continue
    df_small = df_piv[(df_piv.Narrative.str.contains(asset_code)) & (df_piv['Asset Code'] != 'CASH')]
    df_mini = pd.pivot_table(pd.DataFrame(df_small.to_records()), index='Date', values=['Units', 'Unitary Value'], aggfunc={'Units': sum, 'Unitary Value': np.mean})
    df_mini = df_mini[abs(df_mini.Units) > 0.0001]
    try:
        tax_tbl = nutmeg.get_taxable_event_from_single_asset(df_mini)
        tax_tbl['Asset Code'] = asset_code
        tax_tables.append(tax_tbl)
    except Exception as e:
        print(f"Failed to process: {asset_code}: {e}")
tax_tbl = pd.concat(tax_tables).sort_index()

In [ ]:
tax_2022 = tax_tbl[(tax_tbl.index <= '2022-12-01') & (tax_tbl.index > '2021-12-01')]
tax_2022 = pd.DataFrame(tax_2022.to_records())
tax_2022.shape

In [ ]:
def adjust_for_fees(tax_df, fee:float):
    tax_df['fee'] = fee/len(tax_df)
    return tax_df

In [ ]:
fee_for_avg_holding_period = df_piv[(df_piv.Type == 'FEE') & (df_piv.index <= '2022-12-01') & (df_piv.index > '2021-6-01')].Value.sum()
avg_fx = gbpusd_close['2021-6-01':'2022-12-01'].mean()
fee_for_avg_holding_period = fee_for_avg_holding_period * (avg_fx if IN_USD else 1.0)

In [ ]:
tax_2022 = adjust_for_fees(tax_2022, fee_for_avg_holding_period)

In [ ]:
tax_2022_small = pd.pivot_table(tax_2022, index='Asset Code', values=['units_sold', 'sale_price', 'purchase_price', 'purchase_date', 'taxable_amount', 'holding_period', 'fee'],\
         aggfunc={'units_sold':sum, 'sale_price':np.mean, 'purchase_price':np.mean, 'taxable_amount':sum, 'holding_period':np.mean, 'fee': sum})

tax_2022_small.holding_period = tax_2022_small.holding_period.round('1D')
tax_2022_small.taxable_amount = tax_2022_small.taxable_amount + tax_2022_small.fee
tax_2022_small = tax_2022_small.astype({'taxable_amount': 'float', 'units_sold': 'float'}).round(2)
tax_2022_small = tax_2022_small[['units_sold', 'purchase_price', 'sale_price', 'taxable_amount', 'holding_period']]
print(f'Total Capital Gain in 2022 ({"USD" if IN_USD else "GBP"}): {tax_2022_small.taxable_amount.sum():,.2f}')


In [ ]:
tax_2022[tax_2022['Asset Code'] == ''].to_csv(f'{"USD" if IN_USD else "GBP"}_taxable_amounts_details_.csv', index=False)

In [ ]:
tax_2022_small.to_csv(f'{"USD" if IN_USD else "GBP"}_taxable_amounts_at_asset.csv')

In [ ]:
tax_2022_small

In [ ]:
df_plot = df_piv[(df_piv.Narrative.str.contains('')) & (df_piv['Asset Code'] != 'CASH')]

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(df_plot['Unitary Value'], 'gx-')
ax2.bar(x=df_plot['Units'].index, height=list(df_plot['Units']), width=10)

ax1.set_xlabel('X data')
ax1.set_ylabel('Y1 data', color='g')
ax2.set_ylabel('Y2 data', color='b')

plt.grid()
plt.show()